# TP 2 Python et Tkinter
## R309 – Programmation Evenementielle


#### Exercice 1 : Réalisation d’un outil pour dessiner des réseaux

In [2]:
import tkinter as tk
from tkinter import simpledialog
from PIL import Image, ImageTk  


class ApplicationDessinReseau:
    def __init__(self, racine):
        self.racine = racine
        self.racine.title("Outil de Dessin de Réseaux")

        self.canvas = tk.Canvas(racine, bg="white", width=800, height=600)
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

        self.elements_reseau = []
        self.cables = []
        self.element_courant = None
        self.suppression_active = False
        self.creer_cable_active = False
        self.point_depart_cable = None

        self.initialiser_menu()
        self.initialiser_evenements()

        self.charger_images()  # Charger les images


    def charger_images(self):

        image_client = Image.open("/TP2_rendu/PhotoTP/image_routeur.png")
        image_switch = Image.open("/TP2_rendu/PhotoTP/image_switch.png")
        image_routeur = Image.open("/TP2_rendu/PhotoTP/image_routeur.png")

        self.image_client = ImageTk.PhotoImage(image_client)
        self.image_switch = ImageTk.PhotoImage(image_switch)
        self.image_routeur = ImageTk.PhotoImage(image_routeur)


    def initialiser_menu(self):
        self.barre_menu = tk.Menu(self.racine)
        self.racine.config(menu=self.barre_menu)

        menu_fichier = tk.Menu(self.barre_menu, tearoff=0)
        self.barre_menu.add_cascade(label="Fichier", menu=menu_fichier)
        menu_fichier.add_command(label="Quitter", command=self.racine.destroy)

        menu_element = tk.Menu(self.barre_menu, tearoff=0)
        self.barre_menu.add_cascade(label="Élément", menu=menu_element)
        menu_element.add_command(label="Nouveau Client", command=lambda: self.creer_element("Client"))
        menu_element.add_command(label="Nouveau Switch", command=lambda: self.creer_element("Switch"))
        menu_element.add_command(label="Nouveau Routeur", command=lambda: self.creer_element("Routeur"))
        menu_element.add_command(label="Nouveau Câble", command=self.preparer_creation_cable)

        menu_action = tk.Menu(self.barre_menu, tearoff=0)
        self.barre_menu.add_cascade(label="Action", menu=menu_action)
        menu_action.add_command(label="Supprimer", command=self.preparer_suppression)

    def initialiser_evenements(self):
        self.canvas.bind("<Button-1>", self.sur_clic_canvas)
        self.canvas.bind("<B1-Motion>", self.sur_deplacement_canvas)
        self.canvas.bind("<Button-3>", self.afficher_proprietes_element)

    def creer_element(self, type_element):
        x, y = self.obtenir_position_aleatoire()
        element = {
            "type": type_element,
            "id": len(self.elements_reseau) + 1,
            "nom": f"{type_element} {len(self.elements_reseau) + 1}",
            "icone": None,
            "texte": None
        }
        element["icone"], element["texte"] = self.dessiner_element(x, y, type_element, element["nom"])
        self.elements_reseau.append(element)
        self.canvas.tag_bind(element["icone"], "<Button-1>", lambda event, id_element=element["icone"]: self.sur_clic_canvas(event, id_element))

    def dessiner_element(self, x, y, type_element, nom):
        if type_element == "Client":
            icone = self.canvas.create_rectangle(x - 20, y - 20, x + 20, y + 20, fill="blue", tags="element")
        elif type_element == "Switch":
            icone = self.canvas.create_rectangle(x - 20, y - 20, x + 20, y + 20, fill="purple", tags="element")
        elif type_element == "Routeur":
            icone = self.canvas.create_rectangle(x - 20, y - 20, x + 20, y + 20, fill="red", tags="element")
        else:
            return None, None

        # Modifier la ligne suivante pour spécifier la couleur du texte (par exemple, "black" pour noir)
        texte = self.canvas.create_text(x, y + 30, text=nom, tags="element", fill="black")

        return icone, texte

    def sur_clic_canvas(self, event):
        x, y = event.x, event.y
        element = self.canvas.find_closest(x, y)
        if element:
            id_element = element[0]

            if self.creer_cable_active:
                if self.point_depart_cable is None:
                    self.point_depart_cable = {"id": id_element, "x": x, "y": y}
                else:
                    self.dessiner_cable(self.point_depart_cable, {"id": id_element, "x": x, "y": y})
                    self.point_depart_cable = None
                    self.creer_cable_active = False
            else:
                self.deselectionner_element()
                self.element_courant = {"id": id_element, "x": x, "y": y}
                self.canvas.itemconfig(self.element_courant["id"], outline="black", width=2)

    def sur_deplacement_canvas(self, event):
        if self.element_courant:
            dx, dy = event.x - self.element_courant["x"], event.y - self.element_courant["y"]
            self.canvas.move(self.element_courant["id"], dx, dy)
            texte_id = self.obtenir_texte_par_id(self.element_courant["id"])
            if texte_id:
                self.canvas.move(texte_id, dx, dy)
            self.element_courant["x"], self.element_courant["y"] = event.x, event.y
            # Mettre à jour les câbles connectés à l'élément en mouvement
            self.mettre_a_jour_cables(self.element_courant["id"])

    def afficher_proprietes_element(self, event):
        x, y = event.x, event.y
        element = self.canvas.find_closest(x, y)
        if element:
            id_element = element[0]
            proprietes_element = self.obtenir_element_par_id(id_element)
            if proprietes_element:
                nouveau_nom = simpledialog.askstring("Modifier l'élément", "Entrez le nouveau nom :", initialvalue=proprietes_element["nom"])
                if nouveau_nom:
                    proprietes_element["nom"] = nouveau_nom
                    texte_id = self.obtenir_texte_par_id(id_element)
                    if texte_id:
                        self.canvas.itemconfig(texte_id, text=nouveau_nom)

    def preparer_suppression(self):
        self.suppression_active = True
        for element in self.elements_reseau:
            self.canvas.tag_unbind(element["icone"], "<Button-1>")
            self.canvas.tag_bind(element["icone"], "<Button-1>", lambda event, id_element=element["icone"]: self.supprimer_element_clic(event, id_element))

    def preparer_creation_cable(self):
        self.creer_cable_active = True
        self.point_depart_cable = None
        print("Mode création de câble activé. Cliquez sur deux éléments pour créer un câble.")

    def dessiner_cable(self, point_depart, point_arrivee):
        x1, y1 = self.canvas.coords(point_depart["id"])[0], self.canvas.coords(point_depart["id"])[1]
        x2, y2 = self.canvas.coords(point_arrivee["id"])[0], self.canvas.coords(point_arrivee["id"])[1]
        cable = self.canvas.create_line(x1, y1, x2, y2, fill="black", width=2, tags="cable")
        self.cables.append({"id": cable, "points": [point_depart, point_arrivee]})

    def supprimer_element_clic(self, event, id_element=None):
        if id_element is None:
            x, y = event.x, event.y
            element = self.canvas.find_closest(x, y)
            if element:
                id_element = element[0]

        if self.suppression_active:
            # Supprimer le câble connecté à l'élément à supprimer
            self.supprimer_cable_par_element(id_element)

            texte_id = self.obtenir_texte_par_id(id_element)
            self.canvas.delete(id_element)
            if texte_id:
                self.canvas.delete(texte_id)
            self.elements_reseau = [elem for elem in self.elements_reseau if elem["icone"] != id_element]
            self.deselectionner_element()
            self.suppression_active = False

            for element in self.elements_reseau:
                self.canvas.tag_unbind(element["icone"], "<Button-1>")
                self.canvas.tag_bind(element["icone"], "<Button-1>", lambda ev, id_elem=element["icone"]: self.supprimer_element_clic(ev, id_elem))

    def deselectionner_element(self):
        for element in self.elements_reseau:
            self.canvas.itemconfig(element["icone"], outline="black", width=1)

    def obtenir_position_aleatoire(self):
        return 100, 100

    def obtenir_element_par_id(self, id_element):
        for element in self.elements_reseau:
            if element["icone"] == id_element:
                return element
        return None

    def obtenir_texte_par_id(self, id_element):
        for element in self.elements_reseau:
            if element["icone"] == id_element:
                return element["texte"]
        return None

    def supprimer_cable_par_element(self, id_element):
        # Supprimer le câble connecté à l'élément à supprimer
        cables_a_supprimer = [cable for cable in self.cables if any(point["id"] == id_element for point in cable["points"])]
        for cable in cables_a_supprimer:
            self.canvas.delete(cable["id"])
            self.cables.remove(cable)

    def mettre_a_jour_cables(self, id_element):
        # Mettre à jour les câbles connectés à l'élément en mouvement
        for cable in self.cables:
            for point in cable["points"]:
                if point["id"] == id_element:
                    # Mettre à jour les coordonnées du câble
                    x1, y1 = self.canvas.coords(cable["points"][0]["id"])[0], self.canvas.coords(cable["points"][0]["id"])[1]
                    x2, y2 = self.canvas.coords(cable["points"][1]["id"])[0], self.canvas.coords(cable["points"][1]["id"])[1]
                    self.canvas.coords(cable["id"], x1, y1, x2, y2)

if __name__ == "__main__":
    racine = tk.Tk()
    app = ApplicationDessinReseau(racine)
    racine.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/6t/sdff5ltx6fn1gwdvnhgk77dm0000gn/T/ipykernel_80565/2540834314.py", line 73, in <lambda>
    self.canvas.tag_bind(element["icone"], "<Button-1>", lambda event, id_element=element["icone"]: self.sur_clic_canvas(event, id_element))
                                                                                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: ApplicationDessinReseau.sur_clic_canvas() takes 2 positional arguments but 3 were given
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/6t/sdff5l

Mode création de câble activé. Cliquez sur deux éléments pour créer un câble.


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/6t/sdff5ltx6fn1gwdvnhgk77dm0000gn/T/ipykernel_80565/2540834314.py", line 73, in <lambda>
    self.canvas.tag_bind(element["icone"], "<Button-1>", lambda event, id_element=element["icone"]: self.sur_clic_canvas(event, id_element))
                                                                                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: ApplicationDessinReseau.sur_clic_canvas() takes 2 positional arguments but 3 were given
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/6t/sdff5l

#### Exercice 2 : Des traits bien droits !

#### Exercice 3 : Un client exigeant ...